#Prophetic XRD Example

Here we'll define an example of what Hermes will look like.
The particular example we'll consider is autonomously discovering a phase map of a combinitorial wafer from (simulated) 1D X-Ray Diffraction measurements. 

In [ ]:
#Import the lab-scale diffraction instrument emulator.
from Hermes.instruments import XRD_emulator
'''Emulates a 1D Diffractometer instrument.'''

from Hermes.utils import random_locations
'''Chooses random locations to initiallize active learning campaign.'''

from Hermes.utils import find_new_locations 
'''takes in array of locations where we want measurements, 
and array of measurements, 
and returns the locations we don't have measurements yet.'''

#All the possible sample locations (XY coordinates of the stage where XRD measurments can be taken)
# n x 2 array of coordinates [mm]
test_locations = XRD_emulator.test_locations

#The measurement space (in this example this is the theta-2theta range)
# m x 1 array of 2theta positions of the detector [degrees]
measurement_space = XRD_emulator.measurement_space

#Low level specification

In [ ]:
#Maximum Loops:
Loops = 10

#Randomly initialize the measurements
start_measurements = 5
measured_locations = random_locations(test_locations, start_measurements)
measurements = np.array([])

#Batch size
batch_size = 5

for i in Loops:
    #Find the locations where we want measurements but don't have them yet
    new_locations = find_new_locations(measured_locations, measurements)

    #Measure those locations.
    new_measurements = XRD_emulator.measure(new_locations)

    #Append the new measurements to the list of total measurements
    measurements = np.concatenate((measurements, new_measurements), axis = 1)

    #Cluster the measurments we've taken so far (using thier location information as well)
    '''We don't know a piori how many clusters there should be 
    so we'll need to use a community discovery algorithm. 
    We're trying to discover a phase diagram, so the clusters should be contigous.'''
    clusters, cluster_probabilities = Hermes.ContigousCommunityDiscovery.rb_potts(measured_locations, 
                                                                                  measurements)

    #Classify the whole location space.
    '''We can now us a Gaussian Process for Classification to extrapolate 
    the cluster labels to the rest of the location space.
    We know the uncertainty of those labels, so we can use a Heteroscedastic version
    to propagate that uncertainty information to the predictions.'''
    model = Hermes.Heteroscedastic_GPC.train(measured_locations,
                                             clusters, 
                                             cluster_probabilities)
    
    (predicted_phase_labels, 
     predicted_phase_probabilites) =  model.predict(test_locations)

    #Choose the next points to measure.
    '''Now we can choos the next points to measure based on the pediction uncertainties'''
    next_points = batch_acquire(test_locations, 
                                measured_locations, 
                                predicted_phase_probabilites
                                batch_size)

    #Add the next points
    measured_locations = np.concatenate((measured_locations, next_points), axis=0)

    #Archive the data
     #location to save the data to, and the loop index
     #extract the hyper-parameters of the model and saving them
     #save all the training data, and predictions
    archive(save_directory,
            i,
            model, 
            measured_locations,
            measurements,
            clusters, 
            cluster_probabilities,
            predicted_phase_labels,
            predicted_phase_probabilites,
            next_points)





# High level specification

In [ ]:
import hermes

instrument = hermes.XRD_emulator

# specify how to go from the observations to the predictions
data_pipeline = {
preprocessing_step
clustering_method =  hermes.clustering.some_method
classification_method = hermes.classificaition.some_method
} 
#specify how new points will be selected 
acquistion_method = hermes.acquistion_methods.classification_uncertainty


Loops = 10

#Wrap the whole active learning loop in a function
agent = hermes.active_learining_campaign(instrument, 
                                            data_pipeline,
                                            acquistion_method)
#agents and|or instruments have a simpy flag for asychronous execution

In [ ]:
Breakdown of supervised techiques

Supervised
    Classification
        Heteroscedastic
            GPC
            Random_forests
        Homoscedastic
            GPC
            Random_forests
            ImageNet
    Regression
        Heteroscedastic
            GPR
        Homoscedastic
            GPR
            ALLIGN
